# 1.1 Facebook Comments Scraper
- Please update the following FINAL variables to indicate root data folder and user credentials
- error_log.txt file needs to be created before hand within the root data folder
- Additionally, please follow the folder structure below

```
FacebookScraper/
|-- Data/
|   |-- DollarsAndSense/
|   |-- MortgageConsultancy/
|   |-- Seedly/
|   |-- WokeManSalary/
|   |   |-- comments/
|   |   |   |-- WokeManSalary_comments_[xxx].csv
|   |   |   |-- ...
|   |   |   |-- ...
|   |   |-- WokeManSalary_posts.csv
|   |-- error_log.txt
|-- Facebook_Comments.ipynb
```

In [31]:
# Declaration of FINAL Static variables
ROOT_DATA_FOLDER = './Data/' 
EMAIL = '' # <--- Change to own account when using
PASSWORD = '' # <--- Change to own password when using

# Declaration of Dynamic variables
page_list = [
    'DollarsAndSense',
    'MortgageConsultancy',
    'Seedly',
    'WokeManSalary'
]

In [32]:
from selenium import webdriver # allow launching browser
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from bs4 import BeautifulSoup # to parse HTML objects

import os
import csv
import time
import traceback
import datetime

In [33]:
driver_option = webdriver.ChromeOptions()
driver_option.add_argument('--headless')

# driver_option.add_argument('--incognito')
chromedriver_path = '/Users/alfaried/Google Drive/SMU - Y04S01/IS434 SA/Project/chromedriver' # <--- Change this to your own chromedriver path!

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

def login(driver, url):
    
    # Open the website
    driver.implicitly_wait(5)
    driver.get('https://mbasic.facebook.com' + url)

    # Clicks on "Log In" button
    root_element = driver.find_element_by_id('root')
    login_element = root_element.find_element_by_tag_name('a').click()
    time.sleep(1)

    # Input username
    username_input = driver.find_element_by_name("email")
    username_input.clear()
    username_input.send_keys(EMAIL)

    # Input password
    password_input = driver.find_element_by_name("pass")
    password_input.clear()
    password_input.send_keys(PASSWORD)

    # Logs into Facebook using dummy account
    driver.find_element_by_name("login").click()
    time.sleep(1)
    
    return driver

def get_comments_post(driver, comment_link):  
    comment_list = []
    
    # Get comments
    element = driver.find_element_by_xpath('//*[@id="ufi_' + comment_link.split('=')[1].split('&')[0] + '"]/div')

    comment_content_div = element.get_attribute('innerHTML')
    soup = BeautifulSoup(comment_content_div, 'html.parser')

    comment_chunk = soup.contents[-2]
    comments = comment_chunk.contents
    
    if comment_chunk.get('id') == None:         
        for comment in comments:
            try:             
                # Get comment ID
                comment_id = comment.get('id')
                
                # Get user and contents of comments
                tmp = comment.find_all('div')[1]
                
                comment_user = comment.find('h3').get_text()
                comment_date = comment.find('abbr').get_text()
                comment_text = tmp.get_text()
                
                user_tagging_list = []
                user_tagging_link_list = []
                
                user_taggings = tmp.find_all('a')
                for user_tagging in user_taggings:
                    user_tagging_list.append(user_tagging.get_text())
                    user_tagging_link_list.append(user_tagging['href'])
                
                comment_list.append({
                    'user': comment_user,
                    'text': comment_text,
                    'date': comment_date,
                    'id': comment_id,
                    'user_tagging': user_tagging_list,
                    'user_tagging_link': user_tagging_link_list
                })  

            except:
                # If there's a "View more comments..." link
                if 'see_next' in comment.get('id'):
                    driver.get('https://mbasic.facebook.com' + comment.find('a')['href'])

                    tmp_list = get_comments_post(driver, comment_link)

                    if tmp_list != []:
                        comment_list = comment_list + tmp_list
                        
                # Uncomment to print stacktrace
                # traceback.print_exc()

    return comment_list

def get_comments_photo_posts(driver, comment_link):
    comment_list = []
    
    # Get comments
    comment_element = driver.find_element_by_xpath('//*[@id="ufi_' + comment_link.split('/')[-1] + '"]')
    
    comment_content = comment_element.get_attribute('innerHTML')
    comment_soup = BeautifulSoup(comment_content, 'html.parser')
    
    comment_chunk = comment_soup.contents[0].contents[-2]
    
    if comment_chunk.get('id') == None:  
        comments = comment_chunk.contents

        for comment in comments:
            try:
                # Get comment ID
                comment_id = comment.get('id')
                
                # Get user and contents of comments
                tmp = comment.find_all('div')[1]
                
                comment_user = comment.find('h3').get_text()
                comment_date = comment.find('abbr').get_text()
                comment_text = tmp.get_text()
                user_tagging_list = []
                user_tagging_link_list = []
                
                user_taggings = tmp.find_all('a')
                for user_tagging in user_taggings:
                    user_tagging_list.append(user_tagging.get_text())
                    user_tagging_link_list.append(user_tagging['href'])
                
                comment_list.append({
                    'user': comment_user,
                    'text': comment_text,
                    'date': comment_date,
                    'id': comment_id,
                    'user_tagging': user_tagging_list,
                    'user_tagging_link': user_tagging_link_list
                }) 
                
            except:
                # If there's a "View more comments..." link
                if 'see_next' in comment.get('id'):
                    driver.get('https://mbasic.facebook.com' + comment.find('a')['href'])

                    tmp_list = get_comments_photo_posts(driver, comment_link)

                    if tmp_list != []:
                        comment_list = comment_list + tmp_list
                        
                # Uncomment to print stacktrace
                # traceback.print_exc()
                        
    return comment_list

def read_comments_urls_csv(filepath):   
    url_list = []
    print('Reading ' + filepath.split('/')[-1] + '...')
    print('')
    
    with open(filepath, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if 'url' not in row[-1]:
                url_list.append(row[-1])
            
    return url_list

def write_comments_csv(filepath, comments_data):
    data_size = len(comments_data)
    
    if data_size == 0:
        print('No comments')
        print('')
        return
    
    print(str(data_size) + ' comment(s)')
    print('Writing ' + filepath.split('/')[-1] + '...')
    print('')

    with open(filepath, mode='w') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        # Write Header
        csv_writer.writerow(['User', 'Comment', 'Date', 'Comment_ID', 'User_Tagging', 'User_Tagging_Link'])

        # Write Content
        for comment in comments_data:
            tmp = comment['date'].split(' ')
            dateTime = '/'.join(tmp[:2] + ['19']) + ' ' + tmp[-1]

            csv_writer.writerow([comment['user'], comment['text'],  dateTime, comment['id'], ', '.join(comment['user_tagging']), ', '.join(comment['user_tagging_link'])])
            
def write_errors(page, url):
    print('ERROR with URL: ' + url)
    print('Writing to error_log.txt...')
    print('')
    
    with open(ROOT_DATA_FOLDER + 'error_log.txt', mode='a') as csv_file:
        now = datetime.datetime.now()
        
        # Write Content
        csv_file.write('[' + str(now) + '] ERROR for ' + page + '. URL: ' + url + '\n\n')

In [34]:
for page in page_list:
    url_list = read_comments_urls_csv(ROOT_DATA_FOLDER + page + '/' +  page + '_posts.txt')
    comments_folder = ROOT_DATA_FOLDER + page + '/Comments/'
    
    # Loads files in comment folders
    comment_folder_files = os.listdir(comments_folder)

    for url in url_list: 
        post_id = url.split('=')[1].split('&')[0] if 'story' in url else url.split('/')[-1]
        comments_filename = page + '_comments_' + post_id + '.txt'
        
        # Checks if file comments already exists in folder. If TRUE, skips.
        if comments_filename in comment_folder_files:
            print(post_id + ' already scraped')
            print('Skipping...')
            print('')
            continue
        
        # STARTS scraping comments from posts here
        print('Retrieving comments from ' + url)
        comments_data = None  
        comments_filepath = comments_folder + comments_filename
        
        try:
            # Get webdriver Object
            driver = create_webdriver()

            # Log into Facebook
            driver = login(driver, url)
            
            # Retrieve comments
            if 'story' in url:            
                comments_data = get_comments_post(driver, url)            
            else:
                comments_data = get_comments_photo_posts(driver, url)
            
            # Writes to CSV file
            write_comments_csv(comments_filepath, comments_data)
            
        except:
            # Handles exceptions
            write_errors(page, url)
            
            # Uncomment to print stacktrace
            traceback.print_exc()
                
        finally:
            # Close the Webdriver
            driver.close()
        
    print('Scraping comments for ' + page + ' posts, done')
    print('')

Reading DollarsAndSense_posts.txt...

Retrieving comments from /story.php?story_fbid=3182079765195405&id=386868888049854


/Users/alfaried/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


No comments

Retrieving comments from /story.php?story_fbid=3182609391809109&id=386868888049854
1 comment(s)
Writing DollarsAndSense_comments_3182609391809109.txt...

Retrieving comments from /story.php?story_fbid=3180221452047903&id=386868888049854
15 comment(s)
Writing DollarsAndSense_comments_3180221452047903.txt...

Retrieving comments from /story.php?story_fbid=3179548878781827&id=386868888049854
3 comment(s)
Writing DollarsAndSense_comments_3179548878781827.txt...

Retrieving comments from /story.php?story_fbid=3179059135497468&id=386868888049854
2 comment(s)
Writing DollarsAndSense_comments_3179059135497468.txt...

Retrieving comments from /story.php?story_fbid=3175929855810396&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbid=3172813522788696&id=386868888049854
8 comment(s)
Writing DollarsAndSense_comments_3172813522788696.txt...

Retrieving comments from /story.php?story_fbid=3170083759728339&id=386868888049854
No comments

Retrieving comments from

2 comment(s)
Writing DollarsAndSense_comments_3099598306776885.txt...

Retrieving comments from /story.php?story_fbid=3099131223490260&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbid=3097232263680156&id=386868888049854
3 comment(s)
Writing DollarsAndSense_comments_3097232263680156.txt...

Retrieving comments from /story.php?story_fbid=3096822800387769&id=386868888049854
1 comment(s)
Writing DollarsAndSense_comments_3096822800387769.txt...

Retrieving comments from /story.php?story_fbid=3096419477094768&id=386868888049854
4 comment(s)
Writing DollarsAndSense_comments_3096419477094768.txt...

Retrieving comments from /story.php?story_fbid=3093754430694606&id=386868888049854
1 comment(s)
Writing DollarsAndSense_comments_3093754430694606.txt...

Retrieving comments from /story.php?story_fbid=3091031304300252&id=386868888049854
1 comment(s)
Writing DollarsAndSense_comments_3091031304300252.txt...

Retrieving comments from /story.php?story_fbid=3088291167907599

No comments

Retrieving comments from /story.php?story_fbid=3018691801534203&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbid=3016459348424115&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbid=3016017915134925&id=386868888049854
1 comment(s)
Writing DollarsAndSense_comments_3016017915134925.txt...

Retrieving comments from /story.php?story_fbid=3013342592069124&id=386868888049854
1 comment(s)
Writing DollarsAndSense_comments_3013342592069124.txt...

Retrieving comments from /story.php?story_fbid=3013454955391221&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbid=3013048955431821&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbid=3011316045605112&id=386868888049854
6 comment(s)
Writing DollarsAndSense_comments_3011316045605112.txt...

Retrieving comments from /story.php?story_fbid=3010864645650252&id=386868888049854
No comments

Retrieving comments from /story.php?story_fbi

...

2 comment(s)
Writing Seedly_comments_1202465876608111.txt...

Retrieving comments from /story.php?story_fbid=1202387783282587&id=460639650790741
4 comment(s)
Writing Seedly_comments_1202387783282587.txt...

Retrieving comments from /story.php?story_fbid=1202377753283590&id=460639650790741
6 comment(s)
Writing Seedly_comments_1202377753283590.txt...

Retrieving comments from /story.php?story_fbid=1200967976757901&id=460639650790741
1 comment(s)
Writing Seedly_comments_1200967976757901.txt...

Retrieving comments from /story.php?story_fbid=1202309503290415&id=460639650790741
1 comment(s)
Writing Seedly_comments_1202309503290415.txt...

Retrieving comments from /story.php?story_fbid=1201857123335653&id=460639650790741
1 comment(s)
Writing Seedly_comments_1201857123335653.txt...

Retrieving comments from /story.php?story_fbid=1201687460019286&id=460639650790741
1 comment(s)
Writing Seedly_comments_1201687460019286.txt...

Retrieving comments from /story.php?story_fbid=1201576180030414&id=4

1 comment(s)
Writing Seedly_comments_1186250621562970.txt...

Retrieving comments from /story.php?story_fbid=1191545531033479&id=460639650790741
1 comment(s)
Writing Seedly_comments_1191545531033479.txt...

Retrieving comments from /story.php?story_fbid=1191415891046443&id=460639650790741
7 comment(s)
Writing Seedly_comments_1191415891046443.txt...

Retrieving comments from /story.php?story_fbid=1191341441053888&id=460639650790741
1 comment(s)
Writing Seedly_comments_1191341441053888.txt...

Retrieving comments from /story.php?story_fbid=1191257021062330&id=460639650790741
3 comment(s)
Writing Seedly_comments_1191257021062330.txt...

Retrieving comments from /story.php?story_fbid=1190618104459555&id=460639650790741
2 comment(s)
Writing Seedly_comments_1190618104459555.txt...

Retrieving comments from /story.php?story_fbid=1190612951126737&id=460639650790741
3 comment(s)
Writing Seedly_comments_1190612951126737.txt...

Retrieving comments from /story.php?story_fbid=1190544637800235&id=4

2 comment(s)
Writing Seedly_comments_1180241152163917.txt...

Retrieving comments from /story.php?story_fbid=1180173098837389&id=460639650790741
3 comment(s)
Writing Seedly_comments_1180173098837389.txt...

Retrieving comments from /story.php?story_fbid=1178984702289562&id=460639650790741
1 comment(s)
Writing Seedly_comments_1178984702289562.txt...

Retrieving comments from /story.php?story_fbid=1179658398888859&id=460639650790741
10 comment(s)
Writing Seedly_comments_1179658398888859.txt...

Retrieving comments from /story.php?story_fbid=1179574465563919&id=460639650790741
2 comment(s)
Writing Seedly_comments_1179574465563919.txt...

Retrieving comments from /story.php?story_fbid=1179501365571229&id=460639650790741
2 comment(s)
Writing Seedly_comments_1179501365571229.txt...

Retrieving comments from /story.php?story_fbid=1179132482274784&id=460639650790741
2 comment(s)
Writing Seedly_comments_1179132482274784.txt...

Retrieving comments from /story.php?story_fbid=1178965592291473&id=

2 comment(s)
Writing Seedly_comments_1167213870133312.txt...

Retrieving comments from /story.php?story_fbid=1166753396846026&id=460639650790741
2 comment(s)
Writing Seedly_comments_1166753396846026.txt...

Retrieving comments from /story.php?story_fbid=1166634833524549&id=460639650790741
1 comment(s)
Writing Seedly_comments_1166634833524549.txt...

Retrieving comments from /story.php?story_fbid=1166066973581335&id=460639650790741
5 comment(s)
Writing Seedly_comments_1166066973581335.txt...

Retrieving comments from /story.php?story_fbid=1165911836930182&id=460639650790741
2 comment(s)
Writing Seedly_comments_1165911836930182.txt...

Retrieving comments from /story.php?story_fbid=1165881340266565&id=460639650790741
1 comment(s)
Writing Seedly_comments_1165881340266565.txt...

Retrieving comments from /story.php?story_fbid=1165800056941360&id=460639650790741
2 comment(s)
Writing Seedly_comments_1165800056941360.txt...

Retrieving comments from /story.php?story_fbid=1165394693648563&id=4

1 comment(s)
Writing Seedly_comments_1085330478321652.txt...

Retrieving comments from /story.php?story_fbid=1084141615107205&id=460639650790741
No comments

Retrieving comments from /story.php?story_fbid=1081657268688973&id=460639650790741
1 comment(s)
Writing Seedly_comments_1081657268688973.txt...

Retrieving comments from /story.php?story_fbid=1080271825494184&id=460639650790741
1 comment(s)
Writing Seedly_comments_1080271825494184.txt...

Scraping comments for Seedly posts, done

Reading WokeManSalary_posts.txt...

Retrieving comments from /story.php?story_fbid=2413865385595634&id=2250666538582187
11 comment(s)
Writing WokeManSalary_comments_2413865385595634.txt...

Retrieving comments from /story.php?story_fbid=2407835796198593&id=2250666538582187
9 comment(s)
Writing WokeManSalary_comments_2407835796198593.txt...

Retrieving comments from /story.php?story_fbid=2404744283174411&id=2250666538582187
50 comment(s)
Writing WokeManSalary_comments_2404744283174411.txt...

Retrieving co

In [272]:
# .. Start Buffer ..

In [ ]:
# ..

In [ ]:
# ..

In [ ]:
# ..

In [ ]:
# ..

In [ ]:
# .. End Buffer ..

# 2.1 Testing Section
Methods from aboved portion can be tested and used here, just as long as they are successfully compiled

In [109]:
# for page in page_list[3:]:
#     posts_filepath = ROOT_DATA_FOLDER + page + '/' + page + '_collated_comments_tagging.txt'
#     data = []

#     with open(posts_filepath, mode='r', encoding='ISO-8859-1') as csv_file:
#         csv_reader = csv.reader(csv_file, delimiter=',')  
        
#         headers = next(csv_reader)
#         post_id_index = headers.index('Comment_ID')
        
#         for row in csv_reader:
            
#             if row not in data:                
#                 data.append(row)
#             else:
#                 print(row)

In [72]:
# # Convert CSV file into txt file and remove duplicate row
# for page in page_list:
#     posts_filepath = ROOT_DATA_FOLDER + page + '/' + page + '_posts.csv'
#     data = []
    
#     with open(posts_filepath, mode='r', encoding='ISO-8859-1') as csv_file:
#         csv_reader = csv.reader(csv_file, delimiter=',')  
        
#         headers = next(csv_reader)
#         post_id_index = headers.index('post_id')
#         url_index = headers.index('url')
        
#         data.append(headers)
#         for row in csv_reader:
#             new_row = row[:post_id_index]
            
#             url = row[url_index]
#             if 'story' in url:
#                 new_post_id = row[url_index].split('=')[-2].split('&')[0]
#             else:
#                 new_post_id = row[url_index].split('/')[-1]
            
#             new_row.append(new_post_id)
#             new_row.append(url)
            
#             if row not in data:                
#                 data.append(new_row)
            
#     write_filepath = ROOT_DATA_FOLDER + page + '/' + page + '_posts.txt'
    
#     with open(write_filepath, mode='w', encoding='utf-8') as csv_file:
#         csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
#         # Write Header
#         csv_writer.writerow(data[0])

#         # Write Content
#         for line in data[1:]:
#             csv_writer.writerow(line)

In [271]:
# urls_DollarsAndSense = [
#     '/DollarsAndSenseSG/photos/a.1697485630321500/2774662429270476',
#     '/DollarsAndSenseSG/photos/a.900467096690028/2751670081569711',
#     '/DollarsAndSenseSG/photos/a.1697485630321500/2746837705386282',
#     '/DollarsAndSenseSG/photos/a.1697485630321500/2744233652313354',
#     '/DollarsAndSenseSG/photos/a.900467096690028/2694911627245557',
#     '/DollarsAndSenseSG/photos/a.900467096690028/2688094337927286'
# ]

# urls_ConsultMortgage = [
#     '/consultmortgage/photos/a.1455344844620117/1455344894620112'
# ]

# urls_Seedly = [
#     '/seedlysg/photos/a.470073689847337/1059294577591909',
#     '/seedlysg/photos/a.470073689847337/1143479625840070',
#     '/seedlysg/photos/a.470073689847337/1111985378989495'
# ]


# for url in urls_Seedly:
#     comment_list = None
    
#     try:
#         # Get webdriver Object
#         driver = create_webdriver()

#         # Open the website
#         driver.implicitly_wait(5)
#         driver.get('https://mbasic.facebook.com' + url)

#         # Clicks on "Log In" button
#         root_element = driver.find_element_by_id('root')
#         login_element = root_element.find_element_by_tag_name('a').click()
#         time.sleep(1)

#         # Input username
#         username_input = driver.find_element_by_name("email")
#         username_input.clear()
#         username_input.send_keys(EMAIL)

#         # Input password
#         password_input = driver.find_element_by_name("pass")
#         password_input.clear()
#         password_input.send_keys(PASSWORD)

#         # Logs into Facebook using dummy account
#         driver.find_element_by_name("login").click()
#         time.sleep(1)

#         # Retrieve comments from photo posts
#         comment_list = get_comments_photo_posts(driver, url)
        
#         print(url)
#         print(comment_list)

#     except:
#         traceback.print_exc()

#     finally:
#         # Close the Webdriver
#         driver.close()

In [22]:
# url = '/story.php?story_fbid=3182079765195405&id=386868888049854'
# comments_data = None

# try:
#     # Get webdriver Object
#     driver = create_webdriver()

#     # Open the website
#     driver.implicitly_wait(3)
#     driver.get('https://mbasic.facebook.com' + url)
    
#     # Clicks on "Log In" button
#     root_element = driver.find_element_by_id('root')
#     login_element = root_element.find_element_by_tag_name('a').click()
#     time.sleep(1)

#     # tmp = root_element.get_attribute('innerHTML')
#     # print(tmp)
#     # soup = tmp(tmp, 'html.parser')
#     # print(soup)
    
#     # Input username
#     username_input = driver.find_element_by_name("email")
#     username_input.clear()
#     username_input.send_keys(EMAIL) # <--- input credentials

#     # Input password
#     password_input = driver.find_element_by_name("pass")
#     password_input.clear()
#     password_input.send_keys(PASSWORD) # <--- input password

#     # Logs into Facebook using dummy account
#     driver.find_element_by_name("login").click()
#     time.sleep(1)

#     # Retrieve comments
#     comments_data = get_comments_post(driver, url)

# except:
#     traceback.print_exc()

# finally:
#     # Close the Webdriver
#     driver.close()